In [1]:
from gapipes.gaia import Tap

In [2]:
tap = Tap.from_url("http://gea.esac.esa.int:80/tap-server/tap")

In [3]:
tap.tables

[public.dual, 1 columns,
 public.hipparcos, 78 columns,
 public.hipparcos_newreduction, 43 columns,
 public.hubble_sc, 421 columns,
 public.igsl_source, 43 columns,
 public.igsl_source_catalog_ids, 12 columns,
 public.tycho2, 39 columns,
 gaiadr1.aux_qso_icrf2_match, 12 columns,
 gaiadr1.ext_phot_zero_point, 7 columns,
 gaiadr1.allwise_best_neighbour, 8 columns,
 gaiadr1.allwise_neighbourhood, 6 columns,
 gaiadr1.gsc23_best_neighbour, 8 columns,
 gaiadr1.gsc23_neighbourhood, 6 columns,
 gaiadr1.ppmxl_best_neighbour, 8 columns,
 gaiadr1.ppmxl_neighbourhood, 6 columns,
 gaiadr1.sdss_dr9_best_neighbour, 8 columns,
 gaiadr1.sdss_dr9_neighbourhood, 6 columns,
 gaiadr1.tmass_best_neighbour, 8 columns,
 gaiadr1.tmass_neighbourhood, 6 columns,
 gaiadr1.ucac4_best_neighbour, 8 columns,
 gaiadr1.ucac4_neighbourhood, 6 columns,
 gaiadr1.urat1_best_neighbour, 8 columns,
 gaiadr1.urat1_neighbourhood, 6 columns,
 gaiadr1.cepheid, 18 columns,
 gaiadr1.phot_variable_time_series_gfov, 7 columns,
 gaiad

In [4]:
tap.tables.filter(schema='public')

[public.dual, 1 columns,
 public.hipparcos, 78 columns,
 public.hipparcos_newreduction, 43 columns,
 public.hubble_sc, 421 columns,
 public.igsl_source, 43 columns,
 public.igsl_source_catalog_ids, 12 columns,
 public.tycho2, 39 columns]

In [5]:
tap.tables.filter(table='gaia_source')

[gaiadr1.gaia_source, 57 columns, gaiadr2.gaia_source, 96 columns]

In [9]:
dr2 = tap.tables.filter(schema='gaiadr2', table='gaia_source')[0]
print(dr2.description)

This table has an entry for every Gaia observed source as listed in the
Main Database accumulating catalogue version from which the catalogue
release has been generated. It contains the basic source parameters,
that is only final data (no epoch data) and no spectra (neither final
nor epoch).


In [13]:
dr2.as_table

name,datatype,unit,short_description
str32,str8,str16,str60
solution_id,BIGINT,,Solution Identifier
designation,VARCHAR,,Unique source designation (unique across all Data Releases)
source_id,BIGINT,,Unique source identifier (unique within a particular Data Re
random_index,BIGINT,,Random index used to select subsets
ref_epoch,DOUBLE,yr,Reference epoch
ra,DOUBLE,deg,Right ascension
ra_error,DOUBLE,mas,Standard error of right ascension
dec,DOUBLE,deg,Declination
dec_error,DOUBLE,mas,Standard error of declination


In [15]:
dr2.columns[:5]

[Column(name="solution_id", unit="", description="Solution Identifier"),
 Column(name="designation", unit="", description="Unique source designation (uniqu"...,
 Column(name="source_id", unit="", description="Unique source identifier (unique w"...,
 Column(name="random_index", unit="", description="Random index used to select sub"...,
 Column(name="ref_epoch", unit="yr", description="Reference epoch")]

In [22]:
dr2.columns[3]

Column(name="random_index", unit="", description="Random index used to select sub"...

In [21]:
print(dr2.columns[3])

Column name: random_index
unit: 
desription: Random index used to select subsets


In [23]:
r = tap.query('select top 5 * from gaiadr1.gaia_source')

In [24]:
r

,solution_id,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,...,scan_direction_mean_k4,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_mag,phot_variable_flag,l,b,ecl_lon,ecl_lat
0,1635378410781933568,4166407173800868608,760961326,2015.0,262.989326,3.742595,-10.515530,3.213440,NaN,NaN,...,-43.338066,67,155.481886,1.926352,20.045571,NOT_AVAILABLE,14.130678,12.368568,262.932481,12.751920
1,1635378410781933568,4148089005208761472,842820910,2015.0,267.928335,1.229018,-15.300289,1.129408,NaN,NaN,...,43.816254,69,544.384307,2.818229,18.685006,NOT_AVAILABLE,12.447268,5.788742,267.981542,8.124478
2,1635378410781933568,4148112056229094400,739228543,2015.0,266.560022,0.582452,-15.963665,0.578399,NaN,NaN,...,44.931805,76,2109.457855,6.515567,17.214343,NOT_AVAILABLE,11.200458,6.588702,266.664766,7.435798
3,1635378410781933568,4148035919388377088,844525883,2015.0,267.762664,2.162014,-15.793053,4.029215,NaN,NaN,...,43.997116,67,385.696171,2.876878,19.059157,NOT_AVAILABLE,11.938630,5.678470,267.827940,7.629361
4,1635378410781933568,4148087007979299968,840804044,2015.0,267.708518,0.339342,-15.324788,0.327136,NaN,NaN,...,-44.341362,87,4980.463287,12.385516,16.281596,NOT_AVAILABLE,12.318691,5.959595,267.767790,8.096739


In [27]:
j = tap.query_async('select top 1000 * from gaiadr1.gaia_source;')

AttributeError: type object 'Job' has no attribute 'from_xml'